In [3]:
# %load testgpu.py
import mxnet as mx
from mxnet import gluon, init, nd, autograd
from mxnet.gluon import data as gdata, nn, loss as gloss, utils as gutils
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
import time
from mxnet.initializer import Xavier
from mxnet.gluon import data as gdata, loss as gloss, model_zoo, nn
def try_gpu():
    """If GPU is available, return mx.gpu(0); else return mx.cpu()."""
    try:
        ctx = mx.gpu()
        _ = nd.array([0], ctx=ctx)
    except mx.base.MXNetError:
        ctx = mx.cpu()
    return ctx


def evaluate_accuracy(data_iter, net, ctx=[mx.cpu()]):
    """Evaluate accuracy of a model on the given data set."""
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n = nd.array([0]), 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            y = y.astype('float32')
            X = X.astype('float32')
            X=mx.nd.moveaxis(X,-1,1)
            #print(net(X).argmax(axis=1),y)
            acc_sum += (net(X).argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n

def _get_batch(batch, ctx):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
    return (gutils.split_and_load(features, ctx),
            gutils.split_and_load(labels, ctx), features.shape[0])

def load_data_fashion_mnist(batch_size):
    num_workers = 0 if sys.platform.startswith('win32') else 4
    data=gdata.vision.ImageRecordDataset("dataset2.rec")
    train_iter = gdata.DataLoader(data
       , batch_size, shuffle=True,
        num_workers=num_workers)
    data=gdata.vision.ImageRecordDataset("validation_data_10.rec")
    test_iter = gdata.DataLoader(data
        , batch_size, shuffle=False,
        num_workers=num_workers)
    return train_iter, test_iter

lr, num_epochs, ctx = 0.001, 50, try_gpu()
batch_size=16




In [4]:
#load_model
deserialized_net = gluon.nn.SymbolBlock.imports("hand_pose_detection-symbol.json", ['data'], "hand_pose_detection-0050.params", ctx=ctx)
train_iter, test_iter=load_data_fashion_mnist(batch_size)
test_acc = evaluate_accuracy(test_iter,deserialized_net, ctx)
print(test_acc)


0.7
